In [17]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark import SparkContext

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.datastax.spark:spark-cassandra-connector_2.11:2.4.3 --conf spark.cassandra.connection.host=93.175.29.107,93.175.29.109,93.175.29.110,93.175.29.111,93.175.29.113,93.175.29.114,93.175.29.115,93.175.29.116 pyspark-shell'

SparkContext.getOrCreate().stop()

sc = SparkContext("local", 'videos')

sqlContext = SQLContext(sc)
    
# for key, value in spark.sparkContext.getConf().getAll():
#     print(key)
#     print(value)


In [18]:

# Define the function to load data from Cassandra
def load_data_from_cassandra(keyspace, table):
    df = sqlContext.read \
        .format('org.apache.spark.sql.cassandra') \
        .options(table=table, keyspace=keyspace) \
        .load()
    return df

# List keyspaces to validate connection
def list_keyspaces():
    df = sqlContext.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(table="keyspaces", keyspace="system_schema") \
        .load()
    return df



In [19]:
# Test to see if keyspaces are listed
print("Listing keyspaces:")
df_keyspaces = list_keyspaces()
df_keyspaces.show()



Listing keyspaces:
+--------------------+--------------+--------------------+
|       keyspace_name|durable_writes|         replication|
+--------------------+--------------+--------------------+
|hob2020034_hob202...|          true|[class -> org.apa...|
|          hob2020045|          true|[class -> org.apa...|
|    shkunkov_hw_pubg|          true|[class -> org.apa...|
|   babinolegkeyspace|          true|[class -> org.apa...|
|   msb202017keyspace|          true|[class -> org.apa...|
|             test123|          true|[class -> org.apa...|
|     chebotariv_test|          true|[class -> org.apa...|
|             dungeon|          true|[class -> org.apa...|
|            ttansh28|          true|[class -> org.apa...|
|          astanovaks|          true|[class -> org.apa...|
|           ache_test|          true|[class -> org.apa...|
|  akhmatdianov_aleks|          true|[class -> org.apa...|
|           hob201907|          true|[class -> org.apa...|
|          andresokol|          true|

In [20]:
# Load your data
print("Loading videos data:")
movies = load_data_from_cassandra('fedotov19', 'videos')
movies.show()

Loading videos data:
+--------------------+-------------------+--------------------+--------------------+--------------------+
|            video_id|         added_date|         description|               title|             user_id|
+--------------------+-------------------+--------------------+--------------------+--------------------+
|26462163-14bd-11e...|2014-12-10 00:00:00|Google Cloud Plat...|Webinar | From Ze...|10d5c76c-8767-4db...|
|548b6976-da00-1b1...|2015-03-24 08:00:01|It's time to test...|Webinar: Aaron an...|a753812e-92ab-417...|
|c350b26d-1ce5-104...|2015-04-16 00:00:01|Speaker: Joseph S...|Cassandra Day Atl...|8834b74f-132f-40a...|
|f5bd74b3-f153-180...|2015-03-14 16:00:01|Hillary Clinton (...|Hillary Clinton C...|feb35118-35cb-4a8...|
|26462330-14bd-11e...|2013-02-25 00:00:00|                null|Customer Success ...|10d5c76c-8767-4db...|
|075701fc-dc53-1ee...|2015-03-24 16:00:02|Two religious wom...|Uncensored - Key ...|e7236876-9e06-456...|
|264601a3-14bd-11e...|201

In [21]:
movies.show(10)


+--------------------+-------------------+--------------------+--------------------+--------------------+
|            video_id|         added_date|         description|               title|             user_id|
+--------------------+-------------------+--------------------+--------------------+--------------------+
|26462163-14bd-11e...|2014-12-10 00:00:00|Google Cloud Plat...|Webinar | From Ze...|10d5c76c-8767-4db...|
|548b6976-da00-1b1...|2015-03-24 08:00:01|It's time to test...|Webinar: Aaron an...|a753812e-92ab-417...|
|c350b26d-1ce5-104...|2015-04-16 00:00:01|Speaker: Joseph S...|Cassandra Day Atl...|8834b74f-132f-40a...|
|f5bd74b3-f153-180...|2015-03-14 16:00:01|Hillary Clinton (...|Hillary Clinton C...|feb35118-35cb-4a8...|
|26462330-14bd-11e...|2013-02-25 00:00:00|                null|Customer Success ...|10d5c76c-8767-4db...|
|075701fc-dc53-1ee...|2015-03-24 16:00:02|Two religious wom...|Uncensored - Key ...|e7236876-9e06-456...|
|264601a3-14bd-11e...|2011-12-30 00:00:00|Tyle

In [22]:
from pyspark.sql.functions import col

# Фильтр по дате
filtered_movies = movies.filter(col("added_date") > "2015-01-01")
filtered_movies.show(5)


+--------------------+-------------------+--------------------+--------------------+--------------------+
|            video_id|         added_date|         description|               title|             user_id|
+--------------------+-------------------+--------------------+--------------------+--------------------+
|548b6976-da00-1b1...|2015-03-24 08:00:01|It's time to test...|Webinar: Aaron an...|a753812e-92ab-417...|
|c350b26d-1ce5-104...|2015-04-16 00:00:01|Speaker: Joseph S...|Cassandra Day Atl...|8834b74f-132f-40a...|
|f5bd74b3-f153-180...|2015-03-14 16:00:01|Hillary Clinton (...|Hillary Clinton C...|feb35118-35cb-4a8...|
|075701fc-dc53-1ee...|2015-03-24 16:00:02|Two religious wom...|Uncensored - Key ...|e7236876-9e06-456...|
|fe3c4045-6f37-122...|2015-01-16 22:46:44|Saturday Night Li...|40 Years in the M...|539fd1b2-ff34-42c...|
+--------------------+-------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [23]:
movies.groupBy("user_id").count().show()


+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|2e0895de-75aa-4c9...|    1|
|8fa15562-0806-4a4...|    1|
|a47a9adc-3d11-41d...|    2|
|a284468d-d20c-474...|    1|
|99920863-3d90-47c...|    1|
|9b4b3e28-4f75-471...|    1|
|65f52ac8-3393-463...|    1|
|c7f21b67-4ca9-42b...|    1|
|2725b9a7-6590-466...|    2|
|c639fb7f-c734-431...|    2|
|7fb1616d-f44a-459...|    1|
|efac1032-e155-431...|    1|
|6b18f291-84e0-495...|    1|
|73618326-b3c3-443...|    1|
|0854fed7-1ffc-4d6...|    1|
|51fa3e04-59e2-4ed...|    1|
|6dc2bdd4-5aa9-4f5...|    1|
|ce1c125c-169e-4e0...|    1|
|5d150220-1eb7-46c...|    1|
|66b25618-683a-460...|    2|
+--------------------+-----+
only showing top 20 rows



In [24]:
movies.agg({"added_date": "max", "added_date": "min"}).show()


+-------------------+
|    min(added_date)|
+-------------------+
|2011-02-19 00:00:00|
+-------------------+



In [25]:
movies.groupBy("user_id").count().orderBy("count", ascending=False).show(5)


+--------------------+-----+
|             user_id|count|
+--------------------+-----+
|10d5c76c-8767-4db...|  116|
|fd3f7889-fc0c-43d...|    6|
|10179465-69f1-427...|    6|
|ec8cb82f-dcd8-48f...|    4|
|aecbe17d-e0b6-4ac...|    4|
+--------------------+-----+
only showing top 5 rows



In [27]:
# Загрузка данных из другой таблицы
users = sqlContext.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="users", keyspace="python_demo") \
    .load()

# Внутренний джоин по user_id
result = movies.join(users, movies.user_id == users.id, "inner")
result.select("title", "name").show(5)

Py4JJavaError: An error occurred while calling o546.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 22.0 failed 1 times, most recent failure: Lost task 3.0 in stage 22.0 (TID 305, localhost, executor driver): java.io.IOException: Exception during execution of SELECT "id", "name" FROM "python_demo"."users" WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING: Not enough replicas available for query at consistency LOCAL_ONE (1 required but only 0 alive)
	at com.datastax.spark.connector.rdd.CassandraTableScanRDD.com$datastax$spark$connector$rdd$CassandraTableScanRDD$$fetchTokenRange(CassandraTableScanRDD.scala:351)
	at com.datastax.spark.connector.rdd.CassandraTableScanRDD$$anonfun$17.apply(CassandraTableScanRDD.scala:368)
	at com.datastax.spark.connector.rdd.CassandraTableScanRDD$$anonfun$17.apply(CassandraTableScanRDD.scala:368)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$